In [3]:
import pandas as pd
import glob
import datetime


folder_when_write_detail="2018-06-22"
today_str=str(datetime.datetime.now().date())
read_folder="/home/jian/Projects/Big_Lots/Newspaper/zone_zip_level_output_"+folder_when_write_detail+"/"

# Cost

In [2]:
# Extrac the cost PM from the RecbyStore
RecbyStore_folder="/home/jian/Projects/Big_Lots/Newspaper/RecbyStore/"
file_list=glob.glob(RecbyStore_folder+"*")

cost_df=pd.DataFrame()
check_cost_df=pd.DataFrame()
for file in file_list:
    df=pd.read_excel(file,sheetname=None,dtype=str)
    date_name=file.split("/")[len(file.split("/"))-1].split(" ")[0]+" "+file.split("/")[len(file.split("/"))-1].split(" ")[1]+" "+\
    file.split("/")[len(file.split("/"))-1].split(" ")[2]
    if len(date_name.split(" ")[0])==3:
        date_name=datetime.datetime.strptime(date_name,"%b %d %Y").date()
    else:
        date_name=datetime.datetime.strptime(date_name,"%B %d %Y").date()
    
    # print(str(date_name),df.keys())
    
    try:
        df=df[list(df.keys())[0]]
        df=df[~pd.isnull(df['DMA'])]
        df=df[~pd.isnull(df['PRODUCT ID'])]
        df=df[df['DMA']!='nan']
        df=df[df['PRODUCT ID']!='nan']
        df=df[['PRODUCT ID','STORE','CONFIRM MEDIA CPM','PRINT CPM']].drop_duplicates()
        df['Date']=date_name
    except:
        print(str(date_name))
    cost_df=cost_df.append(df)
    
    
    check_cost_df_app=pd.read_excel(file,sheetname=None,dtype=str)
    check_cost_df_app=check_cost_df_app[list(check_cost_df_app.keys())[0]]
    check_cost_df_app=check_cost_df_app[~pd.isnull(check_cost_df_app['DMA'])]
    check_cost_df_app=check_cost_df_app[~pd.isnull(check_cost_df_app['PRODUCT ID'])]
    check_cost_df_app=check_cost_df_app[check_cost_df_app['DMA']!='nan']
    check_cost_df_app=check_cost_df_app[check_cost_df_app['PRODUCT ID']!='nan']
    check_cost_df_app=check_cost_df_app[['PRODUCT ID','STORE','MEDIA COST','PRINT COST']]
    check_cost_df_app['Date']=date_name
    check_cost_df=check_cost_df.append(check_cost_df_app)
    
cost_df=cost_df.rename(columns={"PRODUCT ID":"productid",'STORE':'storeid',"CONFIRM MEDIA CPM":"media_cpm","PRINT CPM":"print_cpm"})
cost_df['media_cpm']=cost_df['media_cpm'].astype(float)
cost_df['print_cpm']=cost_df['print_cpm'].astype(float)
cost_df['productid']=cost_df['productid'].astype(str)
cost_df['storeid']=cost_df['storeid'].astype(str)
cost_df.shape

check_cost_df['MEDIA COST']=check_cost_df['MEDIA COST'].astype(float)
check_cost_df['PRINT COST']=check_cost_df['PRINT COST'].astype(float)
check_cost_df['cost']=check_cost_df['MEDIA COST']+check_cost_df['PRINT COST']

In [3]:
# Deal with one duplication with different media cost
cost_df_1=cost_df[(cost_df['productid']!=400) | (cost_df['storeid']!=555) | (cost_df['Date']!=datetime.datetime(2017,11,23).date())]
cost_df_2=cost_df[(cost_df['productid']==400) & (cost_df['storeid']==555) & (cost_df['Date']==datetime.datetime(2017,11,23).date())]

cost_df_2=pd.read_excel("/home/jian/Projects/Big_Lots/Newspaper/RecbyStore/November 23 2017 RecByStore 6 STD for BL REPORTS.xlsx",dtype=str)
cost_df_2=cost_df_2[['PRODUCT ID','STORE','CONFIRMED CIRCULATION','MEDIA COST',"PRINT CPM"]]
cost_df_2=cost_df_2[(cost_df_2['PRODUCT ID']=="400") & (cost_df_2['STORE']=="555")]
cost_df_2['STORE']=cost_df_2['STORE'].astype(int)
cost_df_2['PRODUCT ID']=cost_df_2['PRODUCT ID'].astype(int)
cost_df_2['MEDIA COST']=cost_df_2['MEDIA COST'].astype(float)
cost_df_2['PRINT CPM']=cost_df_2['PRINT CPM'].astype(float)
cost_df_2['CONFIRMED CIRCULATION']=cost_df_2['CONFIRMED CIRCULATION'].astype(float)
cost_df_2['media_cpm']=cost_df_2['MEDIA COST'].sum()/cost_df_2['CONFIRMED CIRCULATION'].sum()*1000
cost_df_2=cost_df_2.rename(columns={"PRODUCT ID":"productid",'STORE':'storeid',"PRINT CPM":"print_cpm"})
cost_df_2=cost_df_2[['productid','storeid','media_cpm','print_cpm']].drop_duplicates()
cost_df_2['Date']=datetime.datetime(2017,11,23).date()

cost_df_adjusted=cost_df_1.append(cost_df_2).sort_values(['Date','storeid','productid']).reset_index()
del cost_df_adjusted['index']

In [4]:
cost_df_adjusted.head(2)

,productid,storeid,media_cpm,print_cpm,Date
0,165,1,40.7,14.89,2016-02-07
1,72527,1,38.1,14.89,2016-02-07


# Circ

In [4]:
detail_data=pd.read_csv("/home/jian/Projects/Big_Lots/Newspaper/zone_zip_level_output_2018-06-22/"+"BL_combined newspaper final detailed_JL_"+folder_when_write_detail+".csv",dtype=str)

In [6]:
detail_data[['productid','zoneid']].drop_duplicates().shape

(18488, 2)

In [7]:
detail_data['adjusted_circ_with_list']=detail_data['adjusted_circ_with_list'].astype(float)
detail_data['Date']=detail_data['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d").date())
detail_data=detail_data[['zip_cd','zoneid','productid','Date','storeid','adjusted_circ_with_list']]
detail_data=pd.merge(detail_data,cost_df_adjusted,on=['Date','productid','storeid'],how="left")

# Replace the date for 2017-11-22 with 2017-11-23
# Replace the date for 2016-11-23 with 2016-11-24

detail_data['Date']=detail_data['Date'].replace(datetime.datetime(2017,11,22).date(),datetime.datetime(2017,11,23).date())
detail_data['Date']=detail_data['Date'].replace(datetime.datetime(2016,11,23).date(),datetime.datetime(2016,11,24).date())


In [8]:
nan_cpm=detail_data[pd.isnull(detail_data['media_cpm']) | pd.isnull(detail_data['print_cpm'])]
newspaper_detail_long=detail_data[~pd.isnull(detail_data['media_cpm']) & ~pd.isnull(detail_data['print_cpm'])]
nan_cpm_key=nan_cpm[['Date','productid']].drop_duplicates().reset_index()
cost_df_for_nan_agg=pd.DataFrame()
del nan_cpm_key['index']
for i in range(len(nan_cpm_key)):
    df_i=nan_cpm_key.iloc[i,:].to_frame().T.reset_index()
    del df_i['index']
    cost_df_for_nan=cost_df_adjusted[(cost_df_adjusted['Date']==df_i['Date'][0]) & (cost_df_adjusted['productid']==df_i['productid'][0])]
    cost_df_for_nan=cost_df_for_nan.groupby(['Date','productid','media_cpm','print_cpm'])['storeid'].count().reset_index()
    cost_df_for_nan=cost_df_for_nan.sort_values('storeid',ascending=False).reset_index()
    del cost_df_for_nan['index']
    try:
        cost_df_for_nan=cost_df_for_nan.iloc[0,:].to_frame().T.reset_index()    
        del cost_df_for_nan['index']

        del cost_df_for_nan['storeid']
        cost_df_for_nan_agg=cost_df_for_nan_agg.append(cost_df_for_nan)
    except:
        print(nan_cpm_key['Date'][i],nan_cpm_key['productid'][i])

In [9]:
del nan_cpm['media_cpm']
del nan_cpm['print_cpm']
nan_cpm=pd.merge(nan_cpm,cost_df_for_nan_agg,on=["Date","productid"],how="left")
newspaper_detail_long=newspaper_detail_long.append(nan_cpm).reset_index()
del newspaper_detail_long['index']
newspaper_detail_long=newspaper_detail_long.sort_values(['Date','storeid','productid']).reset_index()
del newspaper_detail_long['index']

In [10]:
newspaper_detail_long['media_cpm']=newspaper_detail_long['media_cpm'].astype(float)
newspaper_detail_long['print_cpm']=newspaper_detail_long['print_cpm'].astype(float)
newspaper_detail_long['cost']=newspaper_detail_long['adjusted_circ_with_list']*(newspaper_detail_long['media_cpm']+newspaper_detail_long['print_cpm'])/1000

In [11]:
detail_data_by_Zip_by_Zone_Event=newspaper_detail_long[['zip_cd','zoneid','productid','Date']].drop_duplicates()
detail_data_by_Zip_by_Zone_Event=detail_data_by_Zip_by_Zone_Event.groupby(['zip_cd','zoneid','productid'])['Date'].count().to_frame().reset_index()
detail_data_by_Zip_by_Zone_Circ=newspaper_detail_long.groupby(['zip_cd','zoneid','productid'])['adjusted_circ_with_list'].sum().to_frame().reset_index()
detail_data_by_Zip_by_Zone_Cost=newspaper_detail_long.groupby(['zip_cd','zoneid','productid'])['cost'].sum().to_frame().reset_index()

df_by_zip_zone=pd.merge(detail_data_by_Zip_by_Zone_Event,detail_data_by_Zip_by_Zone_Circ,on=['zip_cd','zoneid','productid'],how="left")
df_by_zip_zone=pd.merge(df_by_zip_zone,detail_data_by_Zip_by_Zone_Cost,on=['zip_cd','zoneid','productid'],how="left")
df_by_zip_zone['zone_newsp_key']="P_"+df_by_zip_zone['productid']+" | Z_"+df_by_zip_zone['zoneid']



In [12]:
df_by_zip_zone=df_by_zip_zone[['zip_cd','zone_newsp_key','productid','zoneid','Date','cost','adjusted_circ_with_list']]
df_by_zip_zone=df_by_zip_zone.rename(columns={"Date":"Events","adjusted_circ_with_list":"total_circ"})

In [13]:
df_by_zip_zone['zip_cd'].apply(lambda x:type(x)).unique()

array([<class 'str'>], dtype=object)

In [14]:
df_by_zip_zone.shape

(44512, 7)

In [15]:
len(df_by_zip_zone['zone_newsp_key'].unique())

18488

In [16]:
df_by_zip_zone.head(2)

,zip_cd,zone_newsp_key,productid,zoneid,Events,cost,total_circ
0,00000,P_106 | Z_Circ verified May 2017;NEW ZONES Eas...,106,Circ verified May 2017;NEW ZONES East 2;Mon Va...,2,1951.64964,41148.0
1,00000,P_580 | Z_City Zone,580,City Zone,1,737.36000,13000.0


# Sales

In [17]:
loyalty_sales_df=pd.read_csv("/home/jian/Projects/Big_Lots/Loyal_members/sales_of_loyalty_member/2018-06-20/sales_by_location_id_agg_2018-06-20.csv",dtype=str)
loyalty_sales_df['customer_zip_code']=loyalty_sales_df['customer_zip_code'].apply(lambda x: x.zfill(5))
loyalty_sales_df['non_loyalty_sales_zip']=loyalty_sales_df['non_loyalty_sales_zip'].astype(float)
loyalty_sales_df['loyal_sales_zip']=loyalty_sales_df['loyal_sales_zip'].astype(float)
loyalty_sales_df['total_sales_zip']=loyalty_sales_df['loyal_sales_zip']+loyalty_sales_df['non_loyalty_sales_zip']
loyalty_sales_df['loyal_sales_pctg']=loyalty_sales_df['loyal_sales_pctg'].astype(float)





non_loyalty_sales_by_zip=loyalty_sales_df.groupby(['customer_zip_code'])['non_loyalty_sales_zip','loyal_sales_zip','total_sales_zip'].sum().reset_index()
non_loyalty_sales_by_zip=non_loyalty_sales_by_zip.rename(columns={"customer_zip_code":"zip_cd",'non_loyalty_sales_zip':'non_loyalty_sales_46',
                                                                 'loyal_sales_zip':'loyal_sales_46','total_sales_zip':'total_sales_46'})
non_loyalty_sales_by_zip['Loyalty_Ratio']=non_loyalty_sales_by_zip['loyal_sales_46']/non_loyalty_sales_by_zip['total_sales_46']
#Exclude zips which loyalty sales >0.65 and <0 and all sales <0
non_loyalty_sales_by_zip_exclude=non_loyalty_sales_by_zip[(non_loyalty_sales_by_zip['non_loyalty_sales_46']<0) |\
                                                         (non_loyalty_sales_by_zip['loyal_sales_46']<0) |\
                                                         (non_loyalty_sales_by_zip['total_sales_46']<0) |\
                                                         (non_loyalty_sales_by_zip['Loyalty_Ratio']<0) |\
                                                         (non_loyalty_sales_by_zip['Loyalty_Ratio']>0.65)]
non_loyalty_sales_by_zip_include=non_loyalty_sales_by_zip[(non_loyalty_sales_by_zip['non_loyalty_sales_46']>=0) &\
                                                         (non_loyalty_sales_by_zip['loyal_sales_46']>=0) &\
                                                         (non_loyalty_sales_by_zip['total_sales_46']>=0) &\
                                                         (non_loyalty_sales_by_zip['Loyalty_Ratio']>=0) &\
                                                         (non_loyalty_sales_by_zip['Loyalty_Ratio']<=0.65)]

In [18]:
df_by_zip_zone_sales=pd.merge(df_by_zip_zone,non_loyalty_sales_by_zip_include,on='zip_cd',how="left")

In [19]:
zip_circ_analysis=pd.read_csv("/home/jian/Projects/Big_Lots/Newspaper/Analysis/zip_level_information.csv",dtype=str)
zip_circ_analysis['zip_cd']=zip_circ_analysis['zip_cd'].apply(lambda x:x.zfill(5))
zip_exclude_1=zip_circ_analysis[(zip_circ_analysis['zip_cd']=="00000") | (zip_circ_analysis['zip_cd']=="99999")]
zip_exclude_2=zip_circ_analysis[(zip_circ_analysis['Event_Count']=="1") | (zip_circ_analysis['total_circ']=="0")]
zip_exclude=zip_exclude_1.append(zip_exclude_2)[['zip_cd','total_circ']]

In [20]:
df_by_zip_zone_sales_Inclusion=df_by_zip_zone_sales[~df_by_zip_zone_sales['zip_cd'].isin(zip_exclude['zip_cd'])]
df_by_zip_zone_sales_Inclusion=df_by_zip_zone_sales_Inclusion[df_by_zip_zone_sales_Inclusion['total_circ']>0]
df_by_zip_zone_sales_Exclusion_1=df_by_zip_zone_sales[df_by_zip_zone_sales['zip_cd'].isin(zip_exclude['zip_cd'])]
df_by_zip_zone_sales_Exclusion_2=df_by_zip_zone_sales_Inclusion[df_by_zip_zone_sales_Inclusion['total_circ']<=0]
df_by_zip_zone_sales_Exclusion=df_by_zip_zone_sales_Exclusion_1.append(df_by_zip_zone_sales_Exclusion_2)

In [21]:
df_by_zip_zone_sales_Exclusion.head(2)

,zip_cd,zone_newsp_key,productid,zoneid,Events,cost,total_circ,non_loyalty_sales_46,loyal_sales_46,total_sales_46,Loyalty_Ratio
0,00000,P_106 | Z_Circ verified May 2017;NEW ZONES Eas...,106,Circ verified May 2017;NEW ZONES East 2;Mon Va...,2,1951.64964,41148.0,7.106392e+07,60325476.35,1.313894e+08,0.459135
1,00000,P_580 | Z_City Zone,580,City Zone,1,737.36000,13000.0,7.106392e+07,60325476.35,1.313894e+08,0.459135


In [22]:
writer=pd.ExcelWriter(read_folder+"BL_Newspaper_by_Zip_by_Zone_JL_"+today_str+".xlsx",engine="xlsxwriter")
df_by_zip_zone_sales_Inclusion.to_excel(writer,"circ_info_by_zone_zip",index=False)
df_by_zip_zone_sales_Exclusion.to_excel(writer,"circ_info_excluded",index=False)
writer.save()

In [24]:
df_by_zip_zone_sales_Inclusion[df_by_zip_zone_sales_Inclusion['zip_cd']=='54914']

,zip_cd,zone_newsp_key,productid,zoneid,Events,cost,total_circ,non_loyalty_sales_46,loyal_sales_46,total_sales_46,Loyalty_Ratio
28466,54914,P_1305 | Z_ZONE_02,1305,ZONE_02,29,3875.951049,74581.680055,330857.98593,263780.67,594638.65593,0.443598
28467,54914,P_1305 | Z_ZONE_02_SC,1305,ZONE_02_SC,29,1092.020798,21154.057200,330857.98593,263780.67,594638.65593,0.443598
28468,54914,P_433 | Z_ZONE_07,433,ZONE_07,1,48.148539,841.022520,330857.98593,263780.67,594638.65593,0.443598
28469,54914,P_1305 | Z_Zone_02,1305,Zone_02,2,268.020982,5283.283688,330857.98593,263780.67,594638.65593,0.443598
28470,54914,P_1305 | Z_Zone_02_SC,1305,Zone_02_SC,2,82.356067,1623.419418,330857.98593,263780.67,594638.65593,0.443598
